In [ ]:
import numpy as np
import tensorflow as tf
import joblib
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, Model, Sequential
from tensorflow.keras.layers import Input, Dense, GaussianNoise, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

tf.compat.v1.enable_eager_execution()

print(f"TensorFlow Version: {tf.__version__}")
print("Libraries imported.")

TensorFlow Version: 2.20.0
Libraries imported.


In [ ]:
class FeatureSqueezing:
    def __init__(self, bit_depth=5):
        self.bit_depth = bit_depth
        self.max_val = 2 ** self.bit_depth - 1
        
    def __call__(self, x):
        x_int = np.round(x * self.max_val)
        x_squeezed = x_int / self.max_val
        return x_squeezed

def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = GaussianNoise(0.05)(input_layer) 
    encoded = Dense(64)(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Activation('relu')(encoded)
    encoded = Dense(32)(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Activation('relu')(encoded)
    decoded = Dense(64)(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Activation('relu')(decoded)
    decoded = Dense(input_dim, activation='linear')(decoded)
    
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse') # Mean Squared Error
    return autoencoder

print("Defense classes defined.")

Defense classes defined.


In [ ]:
print("Loading clean data for Autoencoder training...")

try:
    X_clean = np.load('X_test_ids.npy')
    print(f"Data loaded. Shape: {X_clean.shape}")
except FileNotFoundError:
    print("Error: 'X_test_ids.npy' not found. Please run the save code in your Baseline Notebook.")
    raise

n_samples = X_clean.shape[0]
n_features = X_clean.shape[1]
X_flat = X_clean.reshape(n_samples, n_features)

print("\nTraining Denoising Autoencoder...")
X_train_ae = X_flat[:50000]

autoencoder = build_autoencoder(n_features)
history = autoencoder.fit(
    X_train_ae, X_train_ae,
    epochs=15,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)

autoencoder.save('ids_autoencoder.h5')
print("✓ Autoencoder trained and saved.")

Loading clean data for Autoencoder training...
Data loaded. Shape: (210876, 78, 1)

Training Denoising Autoencoder...
Epoch 1/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2885 - val_loss: 0.1609
Epoch 2/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1171 - val_loss: 0.1112
Epoch 3/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0922 - val_loss: 0.0985
Epoch 4/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0802 - val_loss: 0.0741
Epoch 5/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0714 - val_loss: 0.0593
Epoch 6/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0650 - val_loss: 0.0449
Epoch 7/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0594 - val_loss: 0.0367
Epoch 8/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0540 - val_loss: 0.0393
Epoch 9/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0502 - val_loss: 0.0380
Epoch 10/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0470 - val_loss: 0.0418
Epoch 11/15
352/352 ━

✓ Autoencoder trained and saved.


In [ ]:
print("Loading baseline classifier...")
classifier = load_model('ids_model.h5')

squeezer = FeatureSqueezing(bit_depth=5)

def apply_combined_defense(x_input):
    x_flat = x_input.reshape(x_input.shape[0], -1)
    x_denoised = autoencoder.predict(x_flat, verbose=0)
    x_squeezed = squeezer(x_denoised)
    
    return x_squeezed.reshape(x_input.shape)

attack_files = {
    'FGSM': ('X_adv_fgsm_ids.npy', 'y_test_fgsm_ids_int.npy'),
    'PGD':  ('X_adv_pgd_ids.npy',  'y_test_pgd_ids_int.npy'),
    'C&W':  ('X_adv_cw_ids.npy',   'y_test_cw_ids_int.npy')
}

print("\n" + "="*60)
print("COMBINED DEFENSE EVALUATION (Autoencoder + Squeezing)")
print("="*60)
print(f"{'Attack':<10} {'No Defense':<15} {'Defended':<15} {'Improvement':<15}")
print("-" * 60)

results = {}

for attack_name, (x_path, y_path) in attack_files.items():
    try:
        if not os.path.exists(x_path):
            print(f"{attack_name:<10} -- Skipped (File not found) --")
            continue
            
        X_adv = np.load(x_path)
        y_true = np.load(y_path)
        y_pred_raw = np.argmax(classifier.predict(X_adv, verbose=0), axis=1)
        acc_raw = accuracy_score(y_true, y_pred_raw) * 100
        
        X_defended = apply_combined_defense(X_adv)
        
        y_pred_def = np.argmax(classifier.predict(X_defended, verbose=0), axis=1)
        acc_def = accuracy_score(y_true, y_pred_def) * 100
        
        improvement = acc_def - acc_raw
        results[attack_name] = (acc_raw, acc_def)
        
        print(f"{attack_name:<10} {acc_raw:<15.2f}% {acc_def:<15.2f}% +{improvement:<15.2f}%")
        
    except Exception as e:
        print(f"Error evaluating {attack_name}: {e}")

print("-" * 60)

Loading baseline classifier...

COMBINED DEFENSE EVALUATION (Autoencoder + Squeezing)
Attack     No Defense      Defended        Improvement    
------------------------------------------------------------
FGSM       62.29          % 77.35          % +15.06          %
PGD        34.43          % 75.33          % +40.90          %
C&W        44.92          % 81.20          % +36.28          %
------------------------------------------------------------


In [ ]:
print("Loading baseline classifier...")
classifier = load_model('ids_model.h5')

squeezer = FeatureSqueezing(bit_depth=5)

def apply_combined_defense(x_input):
    x_flat = x_input.reshape(x_input.shape[0], -1)
    x_denoised = autoencoder.predict(x_flat, verbose=0)
    x_squeezed = squeezer(x_denoised)
    return x_squeezed.reshape(x_input.shape)

attack_files = {
    'FGSM': ('X_adv_fgsm_ids.npy', 'y_test_fgsm_ids_int.npy'),
    'PGD':  ('X_adv_pgd_ids.npy',  'y_test_pgd_ids_int.npy'),
    'C&W':  ('X_adv_cw_ids.npy',   'y_test_cw_ids_int.npy')
}

print("\n" + "="*60)
print("COMBINED DEFENSE EVALUATION (Autoencoder + Squeezing)")
print("="*60)
print(f"{'Attack':<10} {'No Defense':<15} {'Defended':<15} {'Improvement':<15}")
print("-" * 60)

results = {}

for attack_name, (x_path, y_path) in attack_files.items():
    try:
        if not os.path.exists(x_path):
            print(f"{attack_name:<10} -- Skipped (File not found) --")
            continue
            
        X_adv = np.load(x_path)
        y_true = np.load(y_path)

        y_pred_raw = np.argmax(classifier.predict(X_adv, verbose=0), axis=1)
        acc_raw = accuracy_score(y_true, y_pred_raw) * 100
        
        X_defended = apply_combined_defense(X_adv)
        
        y_pred_def = np.argmax(classifier.predict(X_defended, verbose=0), axis=1)
        acc_def = accuracy_score(y_true, y_pred_def) * 100
        
        improvement = acc_def - acc_raw
        results[attack_name] = (acc_raw, acc_def)
        
        print(f"{attack_name:<10} {acc_raw:<15.2f}% {acc_def:<15.2f}% +{improvement:<15.2f}%")
        
    except Exception as e:
        print(f"Error evaluating {attack_name}: {e}")

print("-" * 60)

Loading baseline classifier...

COMBINED DEFENSE EVALUATION (Autoencoder + Squeezing)
Attack     No Defense      Defended        Improvement    
------------------------------------------------------------
FGSM       62.29          % 77.35          % +15.06          %
PGD        34.43          % 75.33          % +40.90          %
C&W        44.92          % 81.20          % +36.28          %
------------------------------------------------------------
